**Data Preprocessing**

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import joblib


In [ ]:

# Load dataset
df = pd.read_csv('../data/Friday-WorkingHours-Afternoon-DDos.csv')

# Clean data
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)

# Feature engineering
features = ['Flow Duration', 'Total Fwd Packets', 'Total Backward Packets',
            'Packet Length Mean', 'Packet Length Std', 'Bwd Packet Length Std']

# Label encoding
df['Label'] = df['Label'].map({'BENIGN': 0, 'DDoS': 1})

# Normalization
scaler = MinMaxScaler()
X = scaler.fit_transform(df[features])
y = df['Label'].values

# Save scaler
joblib.dump(scaler, '../models/scaler.pkl')

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**Model Training**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import joblib

# Random Forest Model
rf = RandomForestClassifier(n_estimators=100, max_depth=10)
rf.fit(X_train, y_train)
joblib.dump(rf, '../models/trained_rf_model.pkl')

# LSTM Model
X_train_reshaped = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))

model = Sequential()
model.add(LSTM(64, input_shape=(1, len(features))))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_reshaped, y_train, epochs=10, batch_size=32)
model.save('../models/trained_lstm_model.h5')